In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Messages_text.csv')
df.head()

,sender_id,message
0,-1001285729190,My Visa experience - 07/08/2021\nApproved✅\nDe...
1,-1001285729190,9th July 2021\nLocation: Hyderabad\nAppointmen...
2,-1001285729190,July 9th\nHyderabad Consulate\nIn time 10:25\n...
3,-1001285729190,9th July 2021\nChennai VAC(July 4) and VI\nApp...
4,-1001285729190,Visa experience - 8/7/21 \nApproved✅\nMumbai c...


## Exploring messages

* Its possible that we have empty cells and some jargon messages where the len of the message is < 145 characters.
* Note: 145 is not a magic number. After observing messages where the character size < 145, i concluded that the information available was irrelavent. Hence the messages with < 145 characters are ignored. 

In [3]:
# Check for Nan values in messages column.
df['message'].isnull().sum()
print(df.shape) # (2397, 2)
# As there are 40 null values, we can drop the rows as they are of no use

df.dropna(inplace=True)
print("Shape of dataframe after dropping nan rows")
print(df.shape)

(2397, 2)
Shape of dataframe after dropping nan rows
(2357, 2)


In [4]:

MESSAGES_LEN_TO_IGNORE = 145

df['length_of_message'] = df['message'].apply(lambda x : len(str(x)))
# Filter out of the rows with message length < 145
df_filter = df[df['length_of_message'] > MESSAGES_LEN_TO_IGNORE]


In [5]:
# The final dataframe after filtering out un-necessary messages 
print(df_filter.shape)

(2301, 3)


## Below attributes will be extracted from messages
* [Extracting Status](#extract_status)
* [Extracting Visa Interview Date](#extract_interview_date)
* [Extracting location](#extract_location)
* [Extracting Questions asked in VI](#extract_questions)
* [Extracting University Name](#extract_university)
* ~~Duration~~


### Extracting location
<a id='extract_location'></a>


In [6]:
def get_consulate_location(str_to_check):
    known_consulate_locations = ['hyderabad', 'mumbai', 'kolkata', 'delhi', 'chennai', 'hyd', 'bombay', 'malaysia', 'madras']
    str_converted_to_lower = str_to_check.lower()
    for consulate_location in known_consulate_locations:
        if consulate_location in str_converted_to_lower:
            return consulate_location


df_filter['consulate_location'] = df_filter['message'].apply(get_consulate_location)

/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [7]:
mapping_dict = {'bombay' : "mumbai", 'hyd' : "hyderabad", "madras" : "chennai"}
df_filter['consulate_location'] = df_filter['consulate_location'].apply(lambda x : mapping_dict.get(x) if mapping_dict.get(x) is not None else x )
df_filter['consulate_location'].fillna("NA", inplace=True)

/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [8]:
print(df_filter.consulate_location.value_counts())
df_filter.to_csv("Test.csv", index=False)

mumbai       912
delhi        534
chennai      340
hyderabad    302
kolkata      158
NA            54
malaysia       1
Name: consulate_location, dtype: int64


### Extracting Status 
<a id='extract_status'></a>


In [9]:
def get_visa_status(message):
    possible_status = ['approved', 'rejected']
    for _status in possible_status:
        if _status in message.lower():
            return _status

df_filter['visa_status'] = df_filter['message'].apply(get_visa_status)
df_filter['visa_status'].fillna("NA", inplace=True)


In [10]:
df_filter['visa_status'].value_counts()

approved    2027
rejected     200
NA            74
Name: visa_status, dtype: int64

### Extracting Questions
<a id='extract_questions'></a>


In [22]:
questions_start_with = ['what', 'what\'s', 'which', 'who', 'where', 'why', 'when', 'how', 'whose', 'do', 'are', 'will', 'did ']

import re
import string 


def extract_questions(message):
    questions = []
    regex_pattern = " |".join(questions_start_with)
    for _string in message.lower().split("\n"):
        if _string.endswith("?"):
            questions.append(_string)
        else:
            matches = re.findall(regex_pattern, _string.strip())
            if len(matches) > 0:
                split_str = _string.split()
                if ("vi" in split_str[0] or "vo" in split_str[0]):
                    first_word = split_str[1].strip()
                    if first_word in string.punctuation:
                        for i in range(2, len(split_str)):
                            if split_str[i] not in string.punctuation and split_str[i] not in ['vo', 'vi']:
                                first_word = split_str[i]
                                break


                else:
                     first_word = split_str[0]
                if first_word in questions_start_with:
                    questions.append(_string)
    return questions

df_filter['Questions'] = df_filter['message'].apply(extract_questions)
df_filter['Questions'].fillna("NA", inplace=True)



/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [23]:
df_filter.to_csv("Questions_extracted.csv", index=False)

### Extracting University Name
<a id='extract_university'></a>


In [29]:
# from multiprocessing import  Pool
# from functools import partial
# import numpy as np
# # # Taken from here : https://stackoverflow.com/questions/26784164/pandas-multiprocessing-apply#:~:text=from%20multiprocessing%20import,run_on_subset%2C%20func)%2C%20num_of_processes)

# def parallelize(data, func, num_of_processes=4):
#     data_split = np.array_split(data, num_of_processes)
#     pool = Pool(num_of_processes)
#     data = pd.concat(pool.map(func, data_split))
#     pool.close()
#     pool.join()
#     return data


In [28]:
# df_unv = pd.read_excel('AccreditationData.xlsx', sheet_name='InstituteCampuses')

# def update_parent_data(location_name, parent_name):
#     if parent_name == '-':
#         return location_name
#     else:
#         return parent_name

# df_unv['UniqueName'] = df_unv.apply(lambda x: update_parent_data(x.LocationName, x.ParentName), axis=1)
# unique_university_names = df_unv['UniqueName'].unique()

In [29]:
# print(len(unique_university_names))
# There are 10595 unique universities across USA


10595


In [45]:
# from fuzzywuzzy import fuzz

# matchlist = ['hospital','university','institute','school','academy', 'unv', 'univ']

# unv_regex_str = "|".join(matchlist)

# def get_unv_name_from_text(message):
#     split_str = message.split("\n")
#     for _str in split_str:
#         matches = re.findall(unv_regex_str, _str.strip())
#         if len(matches) > 0:
#             return _str
    
#     # max, max_index = -999999999, "NA"
#     # for unv_index, _unv_name in enumerate(unique_university_names):
#     #     str1, str2 = message, _unv_name
#     #     token_set_ratio = fuzz.token_set_ratio(str1, str2)
#     #     # token_set_ratio_list.append(token_set_ratio)
#     #     if token_set_ratio > max:
#     #         max = token_set_ratio
#     #         max_index = unv_index
#     #         # index = np.argmax(token_set_ratio_list)
#     # try:
#     #     return unique_university_names[max_index]
#     # except Exception as e:
#     return "NA"



In [46]:
# %%time
# from tqdm import tqdm
# tqdm.pandas()

# df_filter["University_name"] = df_filter['message'].progress_apply(get_unv_name_from_text)
# df_filter.to_csv("UnvName_extracted.csv", index=False)

100%|██████████| 2301/2301 [00:00<00:00, 20389.56it/s]
/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
CPU times: user 282 ms, sys: 4 µs, total: 282 ms
Wall time: 273 ms


In [82]:
import spacy
nlp = spacy.load('en')
MONTHDAY = r"(?:(?:0[1-9])|(?:[12][0-9])|(?:3[01])|[1-9])"
MONTH = r"\b(?:jan(?:uary|uar)?|feb(?:ruary|ruar)?|m(?:a|ä)?r(?:ch|z)?|apr(?:il)?|ma(?:y|i)?|jun(?:e|i)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|o(?:c|k)?t(?:ober)?|nov(?:ember)?|de(?:c|z)(?:ember)?)\b"

unv_name = []
visa_interview_date = []

def get_organization_visa_date(message):
    # print(unv_name, visa_interview_date)
    doc = nlp(message)
    final_dict = { entity.text:entity.label_  for entity in doc.ents}
    # print(final_dict)
    visa_date, u_name = None, None
    for key, value in final_dict.items():
        # print(visa_date, u_name)
        if value == 'ORG' and "university" in key.lower() and "research" not in key.lower():
            if u_name == None:
                u_name = key
            else:
                continue
        
        elif value == 'DATE' and re.findall(MONTH, key.lower()) and re.findall(MONTHDAY, key):
            if visa_date == None:
                visa_date = key
            else:
                continue
        else:
            continue
    visa_interview_date.append(visa_date)
    unv_name.append(u_name)



    # visa_date.append(final_dict.get('DATE'))

for row in df_filter.itertuples():
    get_organization_visa_date(row.message)
# df_test['message'].apply(get_organization_visa_date)
df_filter["University_name"] = unv_name
df_filter["VisaInterviewDate"] = visa_interview_date
df_filter.to_csv("FinalData.csv", index=False)


/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### Extracting Interview Date
<a id='extract_interview_date'></a>


In [52]:
%%time
import datefinder

def extract_date_from_message(message):
    try:
        matches = list(datefinder.find_dates(message))
        return matches[0]
    except Exception as e:
        return 'NA'


df_filter['Visa Interview Date'] = df_filter['message'].apply(extract_date_from_message)
df_filter.to_csv("Dates.csv", index=False)

/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname OF identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname M identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/home/ubuntu/repos/f1_visa_interview/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname STAT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this wi

In [53]:
%%time
# from datetime import datetime

# greater_than_date = datetime.strptime('2021-07-12', '%Y-%m-%d')
# less_than_date = datetime.strptime('2020-01-01', '%Y-%m-%d')

# def replace_value(visa_interview_date):
#     try:
#         final_vi_date = datetime.strptime(visa_interview_date.split(" ")[0], '%Y-%m-%d')

#         if (final_vi_date > greater_than_date) or (final_vi_date < less_than_date):
#             return "NA"
#         else:
#             return visa_interview_date
#     except Exception as e:
#         return "NA"


# df_filter['Visa Interview Date'] = df_filter['Visa Interview Date'].apply(replace_value)

# df_filter['Visa Interview Date'] = df_filter['Visa Interview Date'].replace(pd.NaT, "NA")
# df_filter['Visa Interview Date'] = pd.to_datetime(df_filter['Visa Interview Date'])

# df_filter.loc[df_filter['Visa Interview Date'] > greater_than_date, "Visa Interview Date"] = "NA"
# df_filter.loc[df_filter['Visa Interview Date'] < less_than_date, "Visa Interview Date"] = "NA"


# def extract_dates_for_failed_messages(message, extracted_date):
#     try:
#         return dateparser.parse(str(extracted_date))
#     except Exception as e:
#         matches = search_dates(message)
#         for match in matches:
#             if today.month and today.year and today.day:
#                 return match

# df_filter['Visa Interview Date'] = df_filter.apply(lambda x : extract_dates_for_failed_messages(x['message'], x['Visa Interview Date']), axis=1)
df_filter.to_csv("Final_Dates.csv", index=False)           


CPU times: user 102 ms, sys: 20.2 ms, total: 122 ms
Wall time: 119 ms


In [51]:
txt = """ "July 9th
Hyderabad Consulate
In time 10:25
http://t.me/f1interviewreviews
Out time 10:40
University name: University of Connecticut
Status: Approved (45 seconds max)
Appointment time 11:00 AM
Counter 12
VO was a white American lady, super chill and very nicely spoken.
2 other counters were open.

Me: Good morning, Maâ€™am.
VO: Good morning.
VO: Please hold your passport through the screen this way (showed how to)
Me: Held the passport
VO: Can you please pass your I-20 from below the glass?
Me: Passed I-20
VO: When did you graduate?
Me: I graduated in 2017
VO: What did you do since then?
Me: I was working in XXX MNC for the past 3.5 years as an analyst.
VO: Thatâ€™s nice. What are you going to pursue in this University?
Me: I am going to pursue Masters in Business Analytics.
VO (typed for 10 seconds): Why this course?
Me: Told
VO: How are you sponsoring?
Me: Told
VO typed for 10 seconds. Looked at me and typed for another 5-10 seconds.

VO: Take your I-20.
She didnâ€™t speak anything for 5 seconds. I got scared for a while and was looking at her for her reply.
VO: Drop your VISA in the box there. I'm approving your visa.
Me: Thank you so much, Maâ€™am.
VO: Have a good stay at USA. Have fun.
Me: Thank you, Maâ€™am.
She was as excited as I was after approving. Very nicely replied.
@f1interviewreviews"


# """
# import re
# questions = []

# get_unv_name_from_text(txt)



In [52]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load('en')
doc = nlp(txt)
for entity in doc.ents:
    print(entity.text, entity.label_)


July 9th DATE
Hyderabad Consulate PERSON
10:25
http://t.me/f1interviewreviews TIME
10:40 TIME
University of Connecticut ORG
45 seconds max TIME
11:00 AM TIME
American NORP
2 CARDINAL
I-20 PRODUCT
2017 DATE
XXX ORG
the past 3.5 years DATE
™ CARDINAL
Business Analytics ORG
10 seconds TIME
10 seconds TIME
5-10 seconds TIME
I-20 DATE
didnâ€™ LAW
5 seconds TIME
VISA ORG
USA GPE


In [73]:
txt = 'may 7th'
MONTHDAY = r"(?:(?:0[1-9])|(?:[12][0-9])|(?:3[01])|[1-9])"
MONTH = r"\b(?:jan(?:uary|uar)?|feb(?:ruary|ruar)?|m(?:a|ä)?r(?:ch|z)?|apr(?:il)?|ma(?:y|i)?|jun(?:e|i)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|o(?:c|k)?t(?:ober)?|nov(?:ember)?|de(?:c|z)(?:ember)?)\b"
re.findall(MONTHDAY, txt)

['7']